In [1]:
from corpus_reader import read_dataset
#from n_gram_naive_bayes import trained_naive_bayes, extract_features_and_classify

from word_extraction import remove_puntuations
from numpy.random import choice

from sklearn import svm

import sys
import numpy as np

import re

from time import time

In [2]:
data = read_dataset()
n_docs = len(data)

np.random.seed(42)

train_amount = 0.7
train_ids = choice(n_docs, int(train_amount * n_docs), replace=False)
test_ids = [i for i in range(n_docs) if i not in train_ids]

training = [data[i] for i in train_ids]
test = [data[i] for i in test_ids]

print("%i training documents" % len(training))
print("%i test documents" % len(test))


499 training documents
215 test documents


In [39]:
#given a list of document, iterate over the sentences and output a dictionnary counting words occuring
#between drugs and also the number of pairs

class SVM:
    def __init__(self):
        self.nbPairs = 0
        self.count_words = {}
        
        self.nb_feature_words = 20
        self.feature_words = []
        self.feature_index = {}
        
        self.clf = None
        
        
        #given a class, gives the corresponding value
        self.class_index = {
            'null': 0,
            'advise' : 1,
            'effect':2,
            'int':3,
            'mechanism':4
        }
        
        #given a class value, gives the corresponding label
        self.class_mapping = {v: k for k, v in self.class_index.items()}


    def get_interaction_text_list(self, pair):
        text = re.split("\W+", pair.textBetween)
        return " ".join([w for w in text if w != ""])
    
    #count specific words in doc list
    def count_words_doc(self, doc_list, nb=-1, verbose = False, consider_empty_word = True):
        nb = len(doc_list) if nb == -1 else nb
        for doc in doc_list[0:nb]:
            for sentence in doc.sentences:
                if len(sentence.entities) >= 2:
                    self.nbPairs += len(sentence.pairs)
                    all_interaction_text = [self.get_interaction_text_list(p) for p in sentence.pairs]
                    if verbose:
                        print("SENTENCE : " + sentence.text)
                        print("CHECK : " + str(all_interaction_text))
                        print([e.text for e in sentence.entities])
                        print([str(p) for p in sentence.pairs])
                        
                        print("\n-------\n")
                    
                    
                    for interaction_text in all_interaction_text:
                        for w in interaction_text.split(' '):
                            if consider_empty_word or w != "":
                                self.count_words[w] = self.count_words.get(w,0) + 1
  
    
    def create_feature_word_list(self, nb_feature_words = 20):
        srt = sorted([(self.count_words[w], w) for w in self.count_words], reverse = True)
        
        self.nb_feature_words = nb_feature_words
        self.feature_words = [w for (n,w) in srt[:nb_feature_words]]
        
        self.feature_index = {}

        for (i,feat) in enumerate(self.feature_words):
            self.feature_index[feat] = i
    
    def get_feature_from_text(self, wordList):
        output = np.zeros(self.nb_feature_words)
        for w in wordList:
            if w in self.feature_index:
                output[self.feature_index[w]] = 1
        
        return output
    
    def get_features_from_pair(self, pair):
        text = re.split("\W+", pair.textBetween)
        text = " ".join([w for w in text if w != ""])
        
        return self.get_feature_from_text(text)
    
    def create_feature_matrix_and_labels(self,doc_list, nb = -1, verbose = False):
        currPair = 0
        
        ncol = self.nb_feature_words
        nrow = self.nbPairs

        matrixFeature = np.zeros(shape=(nrow, ncol))
        
        labels = [0]*self.nbPairs
        
        nb = len(doc_list) if nb == -1 else nb
        for doc in doc_list[0:nb]:
            for sentence in doc.sentences:
                if len(sentence.entities) >= 2:
                    
                    for p in sentence.pairs:
                        matrixFeature[currPair] = self.get_features_from_pair(p)
                        labels[currPair] = self.class_index[p.getLabel()]
                        currPair += 1
                    
                    if verbose == True:
                        print(sentence.text)
                        print([e.text for e in sentence.entities])
                        #print([get_necessary_info(p, sentence.text) for p in sentence.pairs])
                    
                        
                        #print(all_interaction_text)
                        print("\n-------\n")
        
        if verbose == "minim":
            print("Done.")
        
        return [matrixFeature, labels]
    

In [40]:
psvm = SVM()

In [41]:
psvm.count_words_doc(training)

print("Done")
print("Total number of pairs: " + str(psvm.nbPairs))

Done
Total number of pairs: 20869


In [42]:
psvm.count_words

{'': 1808,
 'limited': 67,
 'iatrogenic': 5,
 'hypercholesterolemic': 6,
 'desflurane': 1,
 'hypercholesterolemia': 3,
 'KRM': 4,
 'implants': 1,
 'all': 25,
 'Elevated': 5,
 'narcotic': 62,
 'four': 18,
 'rapid': 5,
 'sleep': 26,
 'Monoamine': 10,
 'aggression': 1,
 'encainide': 14,
 'HUMIRA': 1,
 'BASELINE': 3,
 'procainamide': 52,
 'physiological': 19,
 'Immediate': 8,
 'colistimethate': 45,
 '340': 1,
 'under': 33,
 'deoxycytidine': 2,
 'dissolution': 4,
 'cytochrome': 42,
 'digit': 4,
 'lack': 2,
 'risk': 64,
 'citalopram': 292,
 'activation': 2,
 'terminal': 4,
 'lincomycin': 48,
 'replication': 1,
 'every': 26,
 'affect': 214,
 'elevations': 9,
 'antiemetics': 6,
 'exaggerate': 3,
 'MAY': 4,
 'induction': 52,
 'procarbazine': 34,
 'Cholestytamine': 12,
 'solution': 24,
 'Phenobarbital': 298,
 'Dose': 6,
 'MAC': 1,
 'THERAPY': 3,
 'ERMBT': 2,
 'protriptyline': 8,
 'cholesterol': 4,
 'indices': 1,
 'enhance': 111,
 'washout': 2,
 'antiretroviral': 7,
 'MAO': 125,
 'nitrofurantoin'

In [43]:
psvm.create_feature_word_list(nb_feature_words = 20)
#print("Showing most "+ str(nb_feature_words) + " frequent words.")

#for e in srt[:nb_feature_words]:
#    print(e)

In [44]:
temp = psvm.create_feature_matrix_and_labels(training)

trainingFeature = temp[0]
labels = np.array(temp[1])

print("Done")

Done


In [45]:
def count_balancing(labels):
    c = {}
    for l in labels:
        c[l] = c.get(l,0) + 1
    
    return c

In [46]:
#SVC (but not NuSVC) implement a keyword class_weight in the fit method.
#It’s a dictionary of the form {class_label : value}, where value is a floating point number > 0
#as it is unbalanced, balance the classes
c = count_balancing(labels)
weight = {i: psvm.nbPairs/float(c[i]) for i in c.keys()}

In [47]:
c

{0: 17937, 1: 599, 2: 1205, 3: 138, 4: 990}

In [48]:
weight

{0: 1.1634610023972793,
 1: 34.83973288814691,
 2: 17.318672199170123,
 3: 151.2246376811594,
 4: 21.07979797979798}

In [49]:


#define the SVM
psvm.clf = svm.LinearSVC()
#psvm.clf = svm.LinearSVC(class_weight=weight)
#psvm.clf = svm.SVC(probability=True, class_weight=weight)


In [50]:
#train it
start = time()
psvm.clf.fit(trainingFeature,labels)
print ("Done in " + str(time() - start) + "s")

Done in 1.55799984932s


# TESTING CLASSIFIERS

In [51]:
#self.class_index = {
#            'null': 0,
#            'advise' : 1,
#            'effect':2,
#            'int':3,
#            'mechanism':4
#        }
correct = 0
total = 0

precision_count = {
    'null': {},
    'advise' : {},
    'effect': {},
    'int': {},
    'mechanism': {}
}

for k in precision_count:
    precision_count[k] = {
        "true_positive" : 0,
        "false_positive" : 0,
        "false_negative" : 0,

        "correct" : 0
    }


In [52]:
#TUNE C !

In [53]:
for tdoc in test:
    for sentence in tdoc.sentences:
        if len(sentence.entities) >= 2:
            for p in sentence.pairs:
                s = p.textBetween
                s = s.lower().split(' ')
                f = psvm.get_feature_from_text(s).reshape(1,-1)
                
                pred = psvm.class_mapping[psvm.clf.predict(f)[0]]
                true = p.getLabel()
                
                if pred == true:
                    precision_count[pred]["true_positive"] += 1
                elif pred != true:
                    precision_count[pred]["false_positive"] += 1
                    precision_count[true]["false_negative"] += 1
                
                total += 1
                
                if pred == true:
                    correct += 1



In [54]:
def print_score(count, label):
    tp = count[label]["true_positive"]
    fp = count[label]["false_positive"]
    fn = count[label]["false_negative"]
    
    print("> %i true positives" % tp)
    print("> %i false positives" % fp)
    print("> %i false negatives" % fn)
    precision = float(tp) / (tp + fp) if tp+fp != 0 else 0
    recall = float(tp) / (tp + fn) if tp+fn != 0 else 0

    print("> recall: %f" % recall)
    print("> precision: %f" % precision)
    print("-"*5)

In [55]:
for k in precision_count:
    print("Precision for class "+k+":")
    print_score(precision_count, k)

Precision for class int:
> 0 true positives
> 0 false positives
> 50 false negatives
> recall: 0.000000
> precision: 0.000000
-----
Precision for class advise:
> 0 true positives
> 0 false positives
> 227 false negatives
> recall: 0.000000
> precision: 0.000000
-----
Precision for class null:
> 5834 true positives
> 1089 false positives
> 0 false negatives
> recall: 1.000000
> precision: 0.842698
-----
Precision for class effect:
> 0 true positives
> 0 false positives
> 483 false negatives
> recall: 0.000000
> precision: 0.000000
-----
Precision for class mechanism:
> 0 true positives
> 0 false positives
> 329 false negatives
> recall: 0.000000
> precision: 0.000000
-----


In [43]:

accuracy = round(100*100*float(correct)/total)/100
print("correct: "+str(correct))
print("total: "+str(total))
print("accuracy: "+str(accuracy)+ "%")


correct: 5834
total: 6923
accuracy: 84.27%
